# Neural Networks

In [1]:
#Dependencies
import keras
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from keras.models import Sequential

In [3]:
from datetime import timedelta
import pandas as pd
import glob

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split


from keras.models import Sequential
from keras.layers import Dense, LeakyReLU, Input


# Load all CSV files in the directory into a list of DataFrames
csv_files = glob.glob('harth2/*.csv')

model = Sequential()
model.add(Dense(12 * 8, input_shape=(306,), activation=LeakyReLU(alpha=0.1)))
model.add(Dense(12 * 4, activation=LeakyReLU(alpha=0.1)))
model.add(Dense(12 * 2, activation=LeakyReLU(alpha=0.1)))
model.add(Dense(13, activation='softmax'))  # Output layer with 12 units

model.compile(loss ='sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

# Initialize lists to store accuracy and predictions
accuracies_NN = []
precisions_NN = []
recalls_NN = []
f1s_NN = []

for csv_file in csv_files:
    # Read the CSV file
    df = pd.read_csv(csv_file)


    lagged = df.copy()
    lagged_columns = []
    
    # Create lagged columns
    lagged_col_data = {}  # Dictionary to store lagged column data
    for i in range(1, 51):
        for col in ['back_x', 'back_y', 'back_z', 'thigh_x', 'thigh_y', 'thigh_z']:
            lagged_col_name = f"{col}_{i}"  # Constructing the lagged column name
            lagged_col_data[lagged_col_name] = lagged[col].shift(i)  # Creating lagged columns
            lagged_columns.append(lagged_col_name)
    
    # Concatenate lagged column data with lagged DataFrame
    lagged = pd.concat([lagged, pd.DataFrame(lagged_col_data)], axis=1)
    
    # Fill NaN values with 0
    lagged.fillna(0, inplace=True)
    
    # Remove the 'label' column from the DataFrame
    label_column = lagged.pop('label')
    
    # Insert the 'label' column at the end
    lagged['label'] = label_column
    
    # Define mapping
    mapping = {1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 13: 9, 14: 10, 130: 11, 140: 12}
    # Replace values
    lagged['label'] = lagged['label'].replace(mapping)
    
    # Display the head of the lagged DataFrame to verify the changes
    lagged.head()
    
    # Drop the 'time_only' column from the lagged DataFrame
    lagged = lagged.drop(['time_only'], axis=1)

    # Split features (X) and labels (y)
    X = lagged.iloc[:, :-1].values  # Exclude the last column, which is the label
    y = lagged.iloc[:, -1].values  # Take only the last column as labels
    
    
    # Normalizing the features
    sc = StandardScaler()
    X = sc.fit_transform(X)
    
    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    """from keras.models import Sequential
    from keras.layers import Dense, LeakyReLU, Input"""
    # Assuming you have determined the total number of unique classes
    # Neural network
    #model = Sequential()
    #model.add(Dense(12 * 8, input_shape=(306,), activation=LeakyReLU(alpha=0.1)))
    #model.add(Dense(12 * 4, activation=LeakyReLU(alpha=0.1)))
    #model.add(Dense(12 * 2, activation=LeakyReLU(alpha=0.1)))
    #model.add(Dense(13, activation='softmax'))  # Output layer with 12 units

    #model.compile(loss ='sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    # Assuming X_train and y_train are prepared from your CSV files
    history = model.fit(X_train, y_train, epochs=50, batch_size=30)

    # Generate generalization metrics
    score = model.evaluate(X_test, y_test, verbose=0)
    print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')
    # Calculate the accuracy for the current file
    accuracy = score[1]  # Assuming score[1] contains the accuracy
    # Append accuracy to the list of accuracies
    accuracies_NN.append(accuracy)

    # predict probabilities for test set
    yhat_probs = model.predict(X_test, verbose=0)
    
    # predict crisp classes for test set
    classes_x=np.argmax(yhat_probs,axis=1)

    # precision tp / (tp + fp)
    precision = precision_score(y_test, classes_x, average='macro')
    print("Precision: %f" % precision)
    precisions_NN.append(precision)
    
    # recall: tp / (tp + fn)
    recall = recall_score(y_test, classes_x, average='macro')
    print("Recall: %f" % recall)
    recalls_NN.append(recall)
    
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(y_test, classes_x, average='macro')
    print("F1 score: %f" % f1)
    f1s_NN.append(f1)

# Calculate the average accuracy,precision,recall,f1 across all files
overall_accuracy = sum(accuracies_NN) / len(accuracies_NN)
print(f'Overall Accuracy: {overall_accuracy}')
overall_precision = sum(precisions_NN)/len(precisions_NN)
print(f'Overall Precision: {overall_accuracy}')
overall_recall = sum(recalls_NN)/len(recalls_NN)
print(f'Overall Recall: {overall_recall}')
overall_f1 = sum(f1s_NN)/len(f1s_NN)
print(f'Overall F1: {overall_f1}')

C:\Users\karag\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
C:\Users\karag\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
10899/10899 ━━━━━━━━━━━━━━━━━━━━ 27s 2ms/step - accuracy: 0.9092 - loss: 0.2621
Epoch 2/50
10899/10899 ━━━━━━━━━━━━━━━━━━━━ 15s 1ms/step - accuracy: 0.9379 - loss: 0.1680
Epoch 3/50
10899/10899 ━━━━━━━━━━━━━━━━━━━━ 16s 1ms/step - accuracy: 0.9448 - loss: 0.1479
Epoch 4/50
10899/10899 ━━━━━━━━━━━━━━━━━━━━ 15s 1ms/step - accuracy: 0.9500 - loss: 0.1361
Epoch 5/50
10899/10899 ━━━━━━━━━━━━━━━━━━━━ 15s 1ms/step - accuracy: 0.9524 - loss: 0.1262
Epoch 6/50
10899/10899 ━━━━━━━━━━━━━━━━━━━━ 14s 1ms/step - accuracy: 0.9560 - loss: 0.1180
Epoch 7/50
10899/10899 ━━━━━━━━━━━━━━━━━━━━ 16s 1ms/step - accuracy: 0.9576 - loss: 0.1132
Epoch 8/50
10899/10899 ━━━━━━━━━━━━━━━━━━━━ 15s 1ms/step - accuracy: 0.9595 - loss: 0.1090
Epoch 9/50
10899/10899 ━━━━━━━━━━━━━━━━━━━━ 16s 1ms/step - accuracy: 0.9608 - loss: 0.1052
Epoch 10/50
10899/10899 ━━━━━━━━━━━━━━━━━━━━ 15s 1ms/step - accuracy: 0.9618 - loss: 0.1012
Epoch 11/50
10899/10899 ━━━━━━━━━━━━━━━━━━━━ 16s 1ms/step - accuracy: 0.9629 - loss: 0.09

C:\Users\karag\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 1/50
9378/9378 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - accuracy: 0.8762 - loss: 3.0406
Epoch 2/50
9378/9378 ━━━━━━━━━━━━━━━━━━━━ 12s 1ms/step - accuracy: 0.9173 - loss: 0.2250
Epoch 3/50
9378/9378 ━━━━━━━━━━━━━━━━━━━━ 21s 1ms/step - accuracy: 0.9252 - loss: 0.1989
Epoch 4/50
9378/9378 ━━━━━━━━━━━━━━━━━━━━ 13s 1ms/step - accuracy: 0.9303 - loss: 0.1804
Epoch 5/50
9378/9378 ━━━━━━━━━━━━━━━━━━━━ 20s 1ms/step - accuracy: 0.9343 - loss: 0.1671
Epoch 6/50
9378/9378 ━━━━━━━━━━━━━━━━━━━━ 13s 1ms/step - accuracy: 0.9380 - loss: 0.1545
Epoch 7/50
9378/9378 ━━━━━━━━━━━━━━━━━━━━ 13s 1ms/step - accuracy: 0.9399 - loss: 0.1496
Epoch 8/50
9378/9378 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.9433 - loss: 0.1405
Epoch 9/50
9378/9378 ━━━━━━━━━━━━━━━━━━━━ 14s 1ms/step - accuracy: 0.9449 - loss: 0.1389
Epoch 10/50
9378/9378 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - accuracy: 0.9463 - loss: 0.1326
Epoch 11/50
9378/9378 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - accuracy: 0.9478 - loss: 0.1285
Epoch 12/50
9378/93

C:\Users\karag\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

C:\Users\karag\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 1/50
8060/8060 ━━━━━━━━━━━━━━━━━━━━ 25s 3ms/step - accuracy: 0.9383 - loss: 0.5167
Epoch 2/50
8060/8060 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.9531 - loss: 0.1632
Epoch 3/50
8060/8060 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.9576 - loss: 0.1292
Epoch 4/50
8060/8060 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.9602 - loss: 0.1140
Epoch 5/50
8060/8060 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - accuracy: 0.9619 - loss: 0.1141
Epoch 6/50
8060/8060 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - accuracy: 0.9627 - loss: 0.1073
Epoch 7/50
8060/8060 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.9647 - loss: 0.0984
Epoch 8/50
8060/8060 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - accuracy: 0.9646 - loss: 0.0954
Epoch 9/50
8060/8060 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - accuracy: 0.9654 - loss: 0.1342
Epoch 10/50
8060/8060 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - accuracy: 0.9670 - loss: 0.0920
Epoch 11/50
8060/8060 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.9663 - loss: 0.0924
Epoch 12/50
8060/80

C:\Users\karag\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 1/50
9003/9003 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - accuracy: 0.8524 - loss: 5.4504
Epoch 2/50
9003/9003 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - accuracy: 0.9065 - loss: 0.3902
Epoch 3/50
9003/9003 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - accuracy: 0.9207 - loss: 0.2566
Epoch 4/50
9003/9003 ━━━━━━━━━━━━━━━━━━━━ 23s 3ms/step - accuracy: 0.9258 - loss: 0.2168
Epoch 5/50
9003/9003 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - accuracy: 0.9305 - loss: 0.1917
Epoch 6/50
9003/9003 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - accuracy: 0.9346 - loss: 0.1765
Epoch 7/50
9003/9003 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - accuracy: 0.9368 - loss: 0.1724
Epoch 8/50
9003/9003 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - accuracy: 0.9377 - loss: 0.1622
Epoch 9/50
9003/9003 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - accuracy: 0.9412 - loss: 0.1670
Epoch 10/50
9003/9003 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - accuracy: 0.9424 - loss: 0.1478
Epoch 11/50
9003/9003 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - accuracy: 0.9437 - loss: 0.1486
Epoch 12/50
9003/90

C:\Users\karag\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
